# Proteome Analysis - norm to total

1. Remove proteins that appear only in one group (only in CC or only in axenic)
2. Impute missing values using low intensity
3. Divide by protein length to normalize by size (optional)
4. Normalize to relative abundance (divide by total abundance) per sample and species 
5. log2 transform
6. compute mean and log2FC between ref and longterm
   (A1, C1 --> C3, C4, C5)
   (P1, C1 --> C3, C4, C5)
7. compute statistics (t-test or Mann-Whitney test)
8. compute FDR

9. goseq - enrichment

 
   




In [1]:
library(DESeq2)
library(RColorBrewer)
library(reshape2)
library("gplots")
library("GGally")                      # Load GGally package
library(goseq)
library(readxl)
library(tidyverse)
library("pheatmap")


Warning message:
"package 'DESeq2' was built under R version 4.2.2"
Loading required package: S4Vectors

Warning message:
"package 'S4Vectors' was built under R version 4.2.2"
Loading required package: stats4

Loading required package: BiocGenerics

Warning message:
"package 'BiocGenerics' was built under R version 4.2.1"

Attaching package: 'BiocGenerics'


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min



Attaching package: 'S4Vectors'


The following objects are masked from 'package:base':

    exp

In [2]:
options(repr.plot.width = 10, repr.plot.height = 10, repr.plot.res = 300)

In [3]:
PRO_COLOR = 'MediumSeaGreen'
ALT_COLOR = 'Gold'
HCOLOR = '#E26A2C'
HCOLOR = '#984215'
 
HCOLOR_light = '#FDA65D'
PCOLOR = '#99cc99'
PCOLOR_light = '#aad5aa'

tpal = brewer.pal(5, "Purples")
gpal = brewer.pal(4, 'Dark2')
ann_colors = list(
    Timepoint = c('1'=tpal[1], '2'=tpal[2], '3'=tpal[3], 
                  '4'=tpal[4], '5'=tpal[5]),
    Group = c(A = HCOLOR_light, C = HCOLOR),
    treatment = c(ref = gpal[2], long.term=gpal[4])
)
#forder = ['med4 (AX)', 'med4 (CC)', '1A3 (CC)', '1A3 (AX)']
#fpal = [PCOLOR_light, PRO_COLOR, HCOLOR, HCOLOR_light,  ]


In [4]:
list.files('data')

[1] "A_axenic_proteome.txt"                                                                          
 [2] "A_axenic_proteome_norm.txt"                                                                     
 [3] "C_ALT_proteome.txt"                                                                             
 [4] "C_ALT_proteome_norm.txt"                                                                        
 [5] "C_PRO_proteome.txt"                                                                             
 [6] "MultiScatter.png"                                                                               
 [7] "Nov22_Processing.xlsx"                                                                          
 [8] "P_axenic_proteome.txt"                                                                          
 [9] "ScatterPlot.png"                                                                                
[10] "uniprot-compressed_true_download_true_fields_accession_2Cid_2Cprotei-2023.01.11-12.43.22.99.txt"

In [5]:
P_df = read_excel(file.path('data', 'Nov22_Processing.xlsx'), sheet = 'MED4', na='NaN', .name_repair = "universal")
A_df = read_excel(file.path('data', 'Nov22_Processing.xlsx'), sheet = 'HOT1A3', na='NaN', .name_repair = "universal")
C_df = read_excel(file.path('data', 'Nov22_Processing.xlsx'), sheet = 'Combined', na='NaN', .name_repair = "universal")

New names:
• `Protein Accession` -> `Protein.Accession`
• `Protein Full Name` -> `Protein.Full.Name`
• `Unique Peptides` -> `Unique.Peptides`
• `Sequence Coverage` -> `Sequence.Coverage`
• `Protein Decoy/Contaminant/Target` -> `Protein.Decoy.Contaminant.Target`
• `Number of Peptides` -> `Number.of.Peptides`
• `Number of Unique Peptides` -> `Number.of.Unique.Peptides`
• `Number of PSMs` -> `Number.of.PSMs`
• `Sequence Coverage Fraction` -> `Sequence.Coverage.Fraction`
New names:
• `Protein Accession` -> `Protein.Accession`
• `Protein Full Name` -> `Protein.Full.Name`
• `Unique Peptides` -> `Unique.Peptides`
• `Sequence Coverage` -> `Sequence.Coverage`
• `Protein Decoy/Contaminant/Target` -> `Protein.Decoy.Contaminant.Target`
• `Number of Peptides` -> `Number.of.Peptides`
• `Number of Unique Peptides` -> `Number.of.Unique.Peptides`
• `Sequence Coverage Fraction` -> `Sequence.Coverage.Fraction`
• `Number of PSMs` -> `Number.of.PSMs`
New names:
• `Protein Accession` -> `Protein.Accession`


In [6]:
P_df = P_df %>%
  pivot_longer(cols = starts_with("Intensity_"), names_to = "Sample", names_prefix = "Intensity_", values_to = "Intensity") #%>%
  #mutate(Sample = str_replace(Sample, "_1", "A")) %>%
  #mutate(Sample = str_replace(Sample, "_2", "B")) %>%
  #mutate(Sample = str_replace(Sample, "_3", "C")) 

C_df = C_df %>%
  pivot_longer(cols = starts_with("Intensity_"), names_to = "Sample", names_prefix = "Intensity_", values_to = "Intensity") #%>%
  #mutate(Sample = str_replace(Sample, "_1", "A")) %>%
  #mutate(Sample = str_replace(Sample, "_2", "B")) %>%
  #mutate(Sample = str_replace(Sample, "_3", "C")) 

A_df = A_df %>%
  pivot_longer(cols = starts_with("Intensity_"), names_to = "Sample", names_prefix = "Intensity_", values_to = "Intensity") #%>%
  #mutate(Sample = str_replace(Sample, "_1", "A")) %>%
  #mutate(Sample = str_replace(Sample, "_2", "B")) %>%
  #mutate(Sample = str_replace(Sample, "_3", "C")) 


In [7]:
mdf = bind_rows(P_df, A_df, C_df)


In [8]:
names(mdf)

[1] "Protein.Accession"                "Gene"                            
 [3] "Organism"                         "Protein.Full.Name"               
 [5] "Unique.Peptides"                  "Sequence.Coverage"               
 [7] "Protein.Decoy.Contaminant.Target" "Number.of.Peptides"              
 [9] "Number.of.Unique.Peptides"        "Number.of.PSMs"                  
[11] "Sequence.Coverage.Fraction"       "Sample"                          
[13] "Intensity"

In [9]:
mdf = mdf %>% 
   extract('Protein.Accession', 
           into=c('protein_id', 'name', 'Species.long'), 
           regex='^([^ ]+)(.*) \\[([^\\]]+)\\]$', 
           remove = FALSE) 


In [10]:
species_map = c(
    'Prochlorococcus marinus' = 'PRO',
    'Alteromonas' = 'ALT', 
    'Alteromonas macleodii' = 'ALT',
    'Prochlorococcus'= 'PRO', 
    'Gammaproteobacteria' = 'ALT',
    'Alteromonas/Salinimonas group' = 'ALT', 
    'Alteromonadaceae' = 'ALT',
    'Cyanobacteria'= 'PRO', 
    'Alteromonas sp. 07-89-2' = 'ALT', 
    'Alteromonadales' = 'ALT'
)


In [11]:
mdf = mdf %>% mutate(Species = species_map[mdf$Species.long])

In [12]:
#mdf %>% select(protein_id, name, Species)

In [13]:
mdf = mdf %>% 
   mutate(Group = substr(mdf$Sample, 1,1)) %>%
   mutate(timepoint = substr(mdf$Sample, 2,2)) %>%
   mutate(replicate = substr(mdf$Sample, 4,4)) %>%
   mutate(Group.timepoint = substr(mdf$Sample, 1,2)) 

In [14]:
cmap = c(A='Axenic', P='Axenic', C='Co-culture')
mdf = mdf %>% 
   mutate(Culture = cmap[Group]) 



In [15]:
mdf = mdf %>% mutate(viable_sample = !(Group.timepoint %in% c('A5', 'P5', 'P3')))

In [16]:
head(mdf)

Protein.Accession                                                       
1 WP_011133152.1 photosystem I core protein PsaA [Prochlorococcus marinus]
2 WP_011133152.1 photosystem I core protein PsaA [Prochlorococcus marinus]
3 WP_011133152.1 photosystem I core protein PsaA [Prochlorococcus marinus]
4 WP_011133152.1 photosystem I core protein PsaA [Prochlorococcus marinus]
5 WP_011133152.1 photosystem I core protein PsaA [Prochlorococcus marinus]
6 WP_011133152.1 photosystem I core protein PsaA [Prochlorococcus marinus]
  protein_id     name                             Species.long            Gene
1 WP_011133152.1  photosystem I core protein PsaA Prochlorococcus marinus NA  
2 WP_011133152.1  photosystem I core protein PsaA Prochlorococcus marinus NA  
3 WP_011133152.1  photosystem I core protein PsaA Prochlorococcus marinus NA  
4 WP_011133152.1  photosystem I core protein PsaA Prochlorococcus marinus NA  
5 WP_011133152.1  photosystem I core protein PsaA Prochlorococcus marinus NA  
6 WP_011133152.1  photosystem I core protein PsaA Prochlorococcus marinus NA  
  Organism Protein.Full.Name
1 NA       NA               
2 NA       NA               
3 NA       NA               
4 NA       NA               
5 NA       NA               
6 NA       NA               
  Unique.Peptides                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [17]:
# look for proteins in both conditions
found_proteins_df = mdf %>% 
  drop_na(Intensity) %>%
  filter(viable_sample) %>% 
  count(Culture, Protein.Accession) %>% 
  count(Protein.Accession) %>%
  filter(n==2)
  


In [18]:
found_proteins_df

Protein.Accession,n
<chr>,<int>
P00760,2
P00761,2
P01031,2
P02533,2
P02538,2
P02768,2
P02769,2
P04264,2
P05787,2


In [19]:
mdf = mdf %>% mutate(found_proteins = Protein.Accession %in% found_proteins_df$Protein.Accession)

In [20]:
head(mdf)

Protein.Accession                                                       
1 WP_011133152.1 photosystem I core protein PsaA [Prochlorococcus marinus]
2 WP_011133152.1 photosystem I core protein PsaA [Prochlorococcus marinus]
3 WP_011133152.1 photosystem I core protein PsaA [Prochlorococcus marinus]
4 WP_011133152.1 photosystem I core protein PsaA [Prochlorococcus marinus]
5 WP_011133152.1 photosystem I core protein PsaA [Prochlorococcus marinus]
6 WP_011133152.1 photosystem I core protein PsaA [Prochlorococcus marinus]
  protein_id     name                             Species.long            Gene
1 WP_011133152.1  photosystem I core protein PsaA Prochlorococcus marinus NA  
2 WP_011133152.1  photosystem I core protein PsaA Prochlorococcus marinus NA  
3 WP_011133152.1  photosystem I core protein PsaA Prochlorococcus marinus NA  
4 WP_011133152.1  photosystem I core protein PsaA Prochlorococcus marinus NA  
5 WP_011133152.1  photosystem I core protein PsaA Prochlorococcus marinus NA  
6 WP_011133152.1  photosystem I core protein PsaA Prochlorococcus marinus NA  
  Organism Protein.Full.Name
1 NA       NA               
2 NA       NA               
3 NA       NA               
4 NA       NA               
5 NA       NA               
6 NA       NA               
  Unique.Peptides                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [21]:
names(mdf)

[1] "Protein.Accession"                "protein_id"                      
 [3] "name"                             "Species.long"                    
 [5] "Gene"                             "Organism"                        
 [7] "Protein.Full.Name"                "Unique.Peptides"                 
 [9] "Sequence.Coverage"                "Protein.Decoy.Contaminant.Target"
[11] "Number.of.Peptides"               "Number.of.Unique.Peptides"       
[13] "Number.of.PSMs"                   "Sequence.Coverage.Fraction"      
[15] "Sample"                           "Intensity"                       
[17] "Species"                          "Group"                           
[19] "timepoint"                        "replicate"                       
[21] "Group.timepoint"                  "Culture"                         
[23] "viable_sample"                    "found_proteins"

In [22]:
fname_hcounts = file.path('..', 'RNASEQ', 'data', 'HOT1A3_counts_combined.txt')
hcdf = read.csv(fname_hcounts, sep='\t' )
hgenes = hcdf[,c(1,2,3,4,5)]
hpath_fname = file.path('..', 'genomes', 'HOT1A3', 'HOT1A3_pathways.csv')
hpath_df = read.csv(hpath_fname)
hpath_df = left_join(hgenes, hpath_df, by='protein_id')

In [23]:
fname_pcounts = file.path('..', 'RNASEQ', 'data', 'MED4_counts_combined.txt')
pcdf = read.csv(fname_pcounts, sep='\t' )
pgenes = pcdf[,c(1,2,3,4,5)]
ppath_fname = file.path('..', 'genomes', 'MED4', 'MED4_pathways.csv')
ppath_df = read.csv(ppath_fname)
ppath_df = left_join(pgenes, ppath_df, by='protein_id')

In [24]:
hpath_df_to_export = hpath_df %>% 
     group_by(gene_id) %>% 
     mutate(
         main = paste0(unique(main), collapse = ";"),
         sub = paste0(unique(sub), collapse = ";"),
         path = paste0(unique(path), collapse = ";"),
         ecpath = paste0(unique(ecpath), collapse = ";")
     ) %>%
    ungroup() %>%
    distinct(gene_id, .keep_all = TRUE)


In [25]:
ppath_df_to_export = ppath_df %>% 
     group_by(gene_id) %>% 
     mutate(
         main = paste0(unique(main), collapse = ";"),
         sub = paste0(unique(sub), collapse = ";"),
         path = paste0(unique(path), collapse = ";"),
         ecpath = paste0(unique(ecpath), collapse = ";")
     ) %>%
    ungroup() %>%
    distinct(gene_id, .keep_all = TRUE)


In [26]:
combined_path_df = bind_rows(ppath_df_to_export, hpath_df_to_export)

In [27]:
names(combined_path_df)

[1] "gene_id"       "gene"          "product"       "protein_id"   
 [5] "old_locus_tag" "X"             "path_id"       "main"         
 [9] "sub"           "path"          "ecpath"

In [28]:
names(mdf)

[1] "Protein.Accession"                "protein_id"                      
 [3] "name"                             "Species.long"                    
 [5] "Gene"                             "Organism"                        
 [7] "Protein.Full.Name"                "Unique.Peptides"                 
 [9] "Sequence.Coverage"                "Protein.Decoy.Contaminant.Target"
[11] "Number.of.Peptides"               "Number.of.Unique.Peptides"       
[13] "Number.of.PSMs"                   "Sequence.Coverage.Fraction"      
[15] "Sample"                           "Intensity"                       
[17] "Species"                          "Group"                           
[19] "timepoint"                        "replicate"                       
[21] "Group.timepoint"                  "Culture"                         
[23] "viable_sample"                    "found_proteins"

In [29]:
#mdf = left_join(mdf, combined_path_df, by = 'protein_id', na_matches='never')

In [31]:
id_cols = c(
    'protein_id', 'name',  'Species', 'Unique.Peptides',
    'Sequence.Coverage', 'Protein.Decoy.Contaminant.Target', 'Number.of.Peptides', 'Number.of.Unique.Peptides', 
    'Number.of.PSMs', 'Sequence.Coverage.Fraction', 
    'found_proteins'#, 'gene_id', 'gene', 'product','path_id','main','sub','path','ecpath'
)
mdf %>% 
   filter(Group %in% 'A', Species %in% 'ALT', found_proteins) %>%
   pivot_wider(id_cols = all_of(id_cols), names_from = Sample, values_from = Intensity) %>%
   write_tsv(file.path('data', 'A_axenic_proteome.txt'))


mdf %>% 
   filter(Group %in% 'P', Species %in% 'PRO', found_proteins) %>%
   pivot_wider(id_cols = all_of(id_cols), names_from = Sample, values_from = Intensity) %>%
   write_tsv(file.path('data', 'P_axenic_proteome.txt'))

mdf %>% 
   filter(Group %in% 'C', Species %in% 'PRO', found_proteins) %>%
   pivot_wider(id_cols = all_of(id_cols), names_from = Sample, values_from = Intensity) %>%
   write_tsv(file.path('data', 'C_PRO_proteome.txt'))

mdf %>% 
   filter(Group %in% 'C', Species %in% 'ALT', found_proteins) %>%
   pivot_wider(id_cols = all_of(id_cols), names_from = Sample, values_from = Intensity) %>%
   write_tsv(file.path('data', 'C_ALT_proteome.txt'))


In [32]:
unique(mdf$name)

[1] " photosystem I core protein PsaA"                                                                                                                                
   [2] " polyribonucleotide nucleotidyltransferase"                                                                                                                      
   [3] " autotransporter outer membrane beta-barrel domain-containing protein"                                                                                           
   [4] " NADP-dependent phosphogluconate dehydrogenase"                                                                                                                  
   [5] " class II fructose-bisphosphatase"                                                                                                                               
   [6] " NAD(P)H-binding protein"                                                                                                                                        
   [7] " chaperonin GroEL"                                                                                                                                               
   [8] " citrate synthase"                                                                                                                                               
   [9] " cell division protein SepF"                                                                                                                                     
  [10] " type I glyceraldehyde-3-phosphate dehydrogenase"                                                                                                                
  [11] " type I glutamate--ammonia ligase"                                                                                                                               
  [12] " cell division protein FtsZ"                                                                                                                                     
  [13] " photosystem II chlorophyll-binding protein CP47"                                                                                                                
  [14] " ATP-dependent zinc metalloprotease FtsH"                                                                                                                        
  [15] " recombinase RecA"                                                                                                                                               
  [16] " bifunctional UDP-N-acetylglucosamine diphosphorylase/glucosamine-1-phosphate N-acetyltransferase GlmU"                                                          
  [17] " DNA-directed RNA polymerase subunit beta'"                                                                                                                      
  [18] " F0F1 ATP synthase subunit beta"                                                                                                                                 
  [19] " MULTISPECIES: form I ribulose bisphosphate carboxylase large subunit"                                                                                           
  [20] " O-acetylhomoserine aminocarboxypropyltransferase/cysteine synthase"                                                                                             
  [21] " DNA-directed RNA polymerase subunit gamma"                                                                                                                      
  [22] " metal ABC transporter substrate-binding protein"                                                                                                                
  [23] " transketolase"                                                                                                                                                  
  [24] " phosphopyruvate hydratase"                                                          